In [ ]:
import pandas as pd
print("Rad模型  结果预览")
df=pd.read_csv('/Users/y2k/workstation_y2k/139fat/run/rad/results/model_performance_metrics.csv')
display(df[df['Model']=='NB'])

print("DL  结果预览")
df=pd.read_csv('/Users/y2k/workstation_y2k/139fat/run/DL/metrics_summary.csv')
display(df[df['Model']=='vit'])

print("clinic_rad模型 MLP算法最佳，结果预览")
df=pd.read_csv('/Users/y2k/workstation_y2k/139fat/run/clinic_rad/results/model_performance_metrics.csv')
display(df[df['Model']=='MLP'])

print("clinic_rad_DL模型 ExtraTree算法最佳，结果预览")
df=pd.read_csv('/Users/y2k/workstation_y2k/139fat/run/clinic_rad_DL/results/model_performance_metrics.csv')
display(df[df['Model']=='ExtraTree'])

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 300
import joblib



In [ ]:

path1='/Users/y2k/workstation_y2k/139fat/run/rad/results/pred'
path2='/Users/y2k/workstation_y2k/139fat/run/clinic_rad/results/pred'
path3='/Users/y2k/workstation_y2k/139fat/run/clinic_rad_DL/results/pred'

# 读取数据
data = {
    'Rad': [f'{path1}/x_train.csv',
            f'{path1}/x_val.csv',
               
                  f'{path1}/x_test.csv',
                    f'{path1}/y_train.csv',
                    f'{path1}/y_val.csv',
    
                     f'{path1}/y_test.csv'
                  ],
    'Clinic_Rad': [f'{path2}/x_train.csv',
              f'{path2}/x_val.csv',
                  f'{path2}/x_test.csv',
                    f'{path2}/y_train.csv',
        f'{path2}/y_val.csv',
                     f'{path2}/y_test.csv'],

    'Clinic_Rad_DL': [f'{path3}/x_train.csv',
              f'{path3}/x_val.csv',
                  f'{path3}/x_test.csv',
                    f'{path3}/y_train.csv',
        f'{path3}/y_val.csv',
                     f'{path3}/y_test.csv'],

}

# 加载模型和预测
models = {
    'Rad': joblib.load(filename='/Users/y2k/workstation_y2k/139fat/run/rad/results/model_weight/NB.pkl'),
    'Clinic_Rad': joblib.load(filename='/Users/y2k/workstation_y2k/139fat/run/clinic_rad/results/model_weight/MLP.pkl'),
     'Clinic_Rad_DL': joblib.load(filename='/Users/y2k/workstation_y2k/139fat/run/clinic_rad_DL/results/model_weight/ExtraTree.pkl'),

}

pred_scores = {}
for key, path_list in data.items():
    pred_scores[key] = {
        'train': models[key].predict_proba(pd.read_csv(path_list[0]).iloc[:,:]),
        'val': models[key].predict_proba(pd.read_csv(path_list[1]).iloc[:,:]),
        'test': models[key].predict_proba(pd.read_csv(path_list[2]).iloc[:,:]),
    }
label_dict = {}
for key, path_list in data.items():
    label_dict[key] = {
        'train': pd.read_csv(path_list[3])['label'],
        'val': pd.read_csv(path_list[4])['label'],
        'test': pd.read_csv(path_list[5])['label'],
    }

predictions={}
for key, path_list in data.items():
    predictions[key] = {
        'train': models[key].predict(pd.read_csv(path_list[0]).iloc[:,:]),
         'val': models[key].predict(pd.read_csv(path_list[1]).iloc[:,:]),
        'test': models[key].predict(pd.read_csv(path_list[2]).iloc[:,:]),
        
    }

In [ ]:
pred_scores['DL']={
    'train': np.array(pd.read_csv('/Users/y2k/workstation_y2k/139fat/clinicdata/DL/train.csv')[['probability_1','probability_1']]),
    'val': np.array(pd.read_csv('/Users/y2k/workstation_y2k/139fat/clinicdata/DL/val.csv')[['probability_1','probability_1']]),

    'test': np.array(pd.read_csv('/Users/y2k/workstation_y2k/139fat/clinicdata/DL/test.csv')[['probability_1','probability_1']])
}

label_dict['DL']={
    'train': np.array(pd.read_csv('/Users/y2k/workstation_y2k/139fat/clinicdata/DL/train.csv')['target']),

    'val': np.array(pd.read_csv('/Users/y2k/workstation_y2k/139fat/clinicdata/DL/val.csv')['target']),
 
    'test': np.array(pd.read_csv('/Users/y2k/workstation_y2k/139fat/clinicdata/DL/test.csv')['target'])
}

## 训练集校准曲线

In [ ]:
from pixelmed_calc.medical_imaging.RadiologyComponents.components1 import HosmerLemeshow
from pixelmed_calc.medical_imaging.Ploting.plot_metric import plot_calibration_curves
model_names = ['Rad', 'DL','Clinic_Rad','Clinic_Rad_DL']

pred_scores_train = [pred_scores[model]['train'][:,1] for model in model_names]
y_test_arrays = [label_dict[model]['train'] for model in model_names]

plot_calibration_curves(y_test_arrays,pred_scores_train,models=model_names,title='Train',n_bins=6)
plt.savefig('results/calibration_curve_train.svg')

In [ ]:
from pixelmed_calc.custom.components.comp1 import draw_calibration
draw_calibration(pred_scores=pred_scores_train, n_bins=5, y_test=y_test_arrays, model_names=model_names,remap=True,title='Train')
plt.savefig('results/calibration_curve_train.svg')

In [ ]:


# 定义要进行hosmer_lemeshow_test的数据标签列表
label_groups =['Rad', 'DL','Clinic_Rad','Clinic_Rad_DL']

hosmer_test_results = {}

for group in label_groups:
    test_labels = label_dict[group]['train']
    test_scores = pred_scores[group]['train'][:,1]


    hl_result_train = HosmerLemeshow(test_scores, test_labels,Q=5)
    print(f'{group} train:\n{hl_result_train}')




In [ ]:
model_names = ['Rad', 'DL','Clinic_Rad','Clinic_Rad_DL']


pred_scores_train = [pred_scores[model]['val'][:,1] for model in model_names]
y_test_arrays = [label_dict[model]['val'] for model in model_names]

plot_calibration_curves(y_test_arrays,pred_scores_train,models=model_names,title='Val')
plt.savefig('results/calibration_curve_val.svg')

In [ ]:
from pixelmed_calc.custom.components.comp1 import draw_calibration
draw_calibration(pred_scores=pred_scores_train, n_bins=5, y_test=y_test_arrays, model_names=model_names,remap=True,title='Train')
plt.savefig('results/calibration_curve_val.svg')

In [ ]:


# 定义要进行hosmer_lemeshow_test的数据标签列表
label_groups =['Rad', 'DL','Clinic_Rad','Clinic_Rad_DL']

hosmer_test_results = {}

for group in label_groups:
    test_labels = label_dict[group]['val']
    test_scores = pred_scores[group]['val'][:,1]


    hl_result_train = HosmerLemeshow(test_scores, test_labels,Q=5)
    print(f'{group} val:\n{hl_result_train}')




In [ ]:
model_names = ['Rad', 'DL','Clinic_Rad','Clinic_Rad_DL']


pred_scores_train = [pred_scores[model]['test'][:,1] for model in model_names]
y_test_arrays = [label_dict[model]['test'] for model in model_names]

plot_calibration_curves(y_test_arrays,pred_scores_train,models=model_names,title='Test')
plt.savefig('results/calibration_curve_test.svg')

In [ ]:
from pixelmed_calc.custom.components.comp1 import draw_calibration
draw_calibration(pred_scores=pred_scores_train, n_bins=5, y_test=y_test_arrays, model_names=model_names,remap=True,title='Train')
plt.savefig('results/calibration_curve_test.svg')

In [ ]:


# 定义要进行hosmer_lemeshow_test的数据标签列表
label_groups =['Rad', 'DL','Clinic_Rad','Clinic_Rad_DL']

hosmer_test_results = {}

for group in label_groups:
    test_labels = label_dict[group]['test']
    test_scores = pred_scores[group]['test'][:,1]


    hl_result_train = HosmerLemeshow(test_scores, test_labels,Q=5)
    print(f'{group} test:\n{hl_result_train}')




In [ ]:
from pixelmed_calc.medical_imaging.Ploting.plot_metric import plot_DCA_all

labels = model_names 

y_label_list_train = [np.array(label_dict[label]['train']) for label in labels]
predlabel_list_train = [pred_scores[label]['train'][:, 1] for label in labels]

# Now call plot_DCA function with the simplified lists
plot_DCA_all(predlabel_list_train, y_label_list_train, labels=labels, title='Model DCA Train', label_loc='lower left',y_min=-1)
plt.savefig('results/DAC-train.svg')
plt.close

In [ ]:
from pixelmed_calc.medical_imaging.Ploting.plot_metric import plot_DCA_all

labels = model_names 

y_label_list_train = [np.array(label_dict[label]['val']) for label in labels]
predlabel_list_train = [pred_scores[label]['val'][:, 1] for label in labels]

# Now call plot_DCA function with the simplified lists
plot_DCA_all(predlabel_list_train, y_label_list_train, labels=labels, title='Model DCA Val', label_loc='lower left',y_min=-1)
plt.savefig('results/DAC-val.svg')
plt.close

In [ ]:


labels = model_names 


y_label_list_train = [np.array(label_dict[label]['test']) for label in labels]
predlabel_list_train = [pred_scores[label]['test'][:, 1] for label in labels]

# Now call plot_DCA function with the simplified lists
plot_DCA_all(predlabel_list_train, y_label_list_train, labels=labels, title='Model DCA Test', label_loc='lower left',y_min=-1)
plt.savefig('results/DAC-test.svg')
plt.close

训练集 ROC

In [ ]:
from pixelmed_calc.medical_imaging.Ploting.plot_metric import plot_multiple_ROCs
pred_scores_train = [pred_scores[model]['train'][:,1] for model in model_names]
y_test_arrays = [np.array(label_dict[model]['train']) for model in model_names]


plot_multiple_ROCs(y_test_arrays,pred_scores_train,models=model_names,title='Train ROC')
plt.savefig(f'results/train_roc.svg')
plt.show()



In [ ]:
from pixelmed_calc.medical_imaging.Ploting.plot_metric import plot_multiple_ROCs
pred_scores_train = [pred_scores[model]['val'][:,1] for model in model_names]
y_test_arrays = [np.array(label_dict[model]['val']) for model in model_names]


plot_multiple_ROCs(y_test_arrays,pred_scores_train,models=model_names,title='Val ROC')
plt.savefig(f'results/val_roc.svg')
plt.show()



测试集ROC

In [ ]:
from pixelmed_calc.medical_imaging.Ploting.plot_metric import plot_multiple_ROCs
pred_scores_train = [pred_scores[model]['test'][:,1] for model in model_names]
y_test_arrays = [np.array(label_dict[model]['test']) for model in model_names]


plot_multiple_ROCs(y_test_arrays,pred_scores_train,models=model_names,title='Test ROC')
plt.savefig(f'results/test_roc.svg')
plt.show()



## delong检验

e-16 代码10的负16次方

In [ ]:
from pixelmed_calc.medical_imaging.RadiologyComponents.delong import delong_roc_test
from itertools import combinations
import numpy as np
# Define your models and dataset types

dataset_types = ['train','val','test']

# Iterate through dataset types and model pairs to perform DeLong tests

for dataset_type in dataset_types:
    for model_one, model_two in combinations(model_names, 2):
        ground_truth = np.array(label_dict[model_two][dataset_type])
        predictions_one = pred_scores[model_one][dataset_type][:, 1]
        predictions_two = pred_scores[model_two][dataset_type][:, 1]

        #print(model_one,model_two,predictions_one.shape,predictions_two.shape)

        delong = delong_roc_test(ground_truth, predictions_one, predictions_two)
        print(f'delong检验-{dataset_type}集: {model_one} vs {model_two}，p值: {delong[0][0]}, z值: {delong[1][0]}')